In [21]:
# install the lxml to read html pages

!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



__Install the required libraries__

In [22]:
#install the required libraries

import numpy as np
import pandas as pd
import lxml

__Read the html page and store the required rows in a dataframe__

In [23]:
# read the html page
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# read the table
df = df[0]

# remove all Not assigned borough rows

df = df[df['Borough'] != 'Not assigned']
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


__Sort the dataframe__

In [24]:
# sort the table on postcode to ensure all same postcode rows comes together for merging

df.sort_values(by='Postcode',inplace=True)
df

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
29,M1C,Scarborough,Port Union
28,M1C,Scarborough,Rouge Hill
27,M1C,Scarborough,Highland Creek
...,...,...,...
232,M9V,Etobicoke,Mount Olive
234,M9V,Etobicoke,South Steeles
235,M9V,Etobicoke,Thistletown
233,M9V,Etobicoke,Silverstone


__Loop through the dataframe and merge the neighbourhood in the rows with same Postcode__

In [20]:
# initialize the data elements to be used in the loop

list = []  # empty list
columns = ['Postcode','Borough','Neighbourhood'] # set the column headers
neigh_list = ''
prev_postcode = df.iloc[0][0]
prev_borough = df.iloc[0][1]
first_row = 'TRUE'

# loop through the entire list

for postcode, borough, neighbourhood in zip(df['Postcode'], df['Borough'], df['Neighbourhood']): 
    if neighbourhood == 'Not assigned': # set Not assigned neighbourhood with Borough names
        neighbourhood = borough
    if postcode == prev_postcode: # compare with previous rows
        if first_row == 'TRUE': # special processing for first row to take care of ,
            neigh_list = neighbourhood
            first_row = 'FALSE'
        else:
            neigh_list = neigh_list + ', ' + neighbourhood # keep appending the neighbourhood to the list
    else: # we are done with all same postcode rows so append to the list
        list.append([prev_postcode,prev_borough,neigh_list])
        neigh_list = neighbourhood
    prev_postcode = postcode # save the current row values
    prev_borough = borough # save the current row values

# write the last row
list.append([postcode,borough,neigh_list])

# convert list to dataframe
df_n = pd.DataFrame(list,columns=columns)
df_n

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Richview Gardens, Kingsview Village, St. Phill..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


__Print the shape of the final dataframe__

In [16]:
df_n.shape

(103, 3)